<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
#Uet_Utils
#Algorithms.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace Uet_Utils
{
    // origin source here
    // https://gist.github.com/Davidblkx/e12ab0bb2aff7fd8072632b396538560
    // Thank you Davidblkx
    internal class Algorithms
    {
        /// <summary>
        ///     Calculate the difference between 2 strings using the Levenshtein distance algorithm
        /// </summary>
        /// <param name="source1">First string</param>
        /// <param name="source2">Second string</param>
        /// <returns></returns>
        public static int Calculate(string source1, string source2) //O(n*m)
        {
            var source1Length = source1.Length;
            var source2Length = source2.Length;

            var matrix = new int[source1Length + 1, source2Length + 1];

            // First calculation, if one entry is empty return full length
            if (source1Length == 0)
                return source2Length;

            if (source2Length == 0)
                return source1Length;

            // Initialization of matrix with row size source1Length and columns size source2Length
            for (var i = 0; i <= source1Length; matrix[i, 0] = i++) { }
            for (var j = 0; j <= source2Length; matrix[0, j] = j++) { }

            // Calculate rows and collumns distances
            for (var i = 1; i <= source1Length; i++)
            {
                for (var j = 1; j <= source2Length; j++)
                {
                    var cost = (source2[j - 1] == source1[i - 1]) ? 0 : 1;

                    matrix[i, j] = Math.Min(
                        Math.Min(matrix[i - 1, j] + 1, matrix[i, j - 1] + 1),
                        matrix[i - 1, j - 1] + cost);
                }
            }
            // return result
            return matrix[source1Length, source2Length];
        }
    }
}


In [ ]:
#Audio_Recognition.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Threading;
using NAudio.Wave;
using Whisper.net;
using System.Diagnostics;

namespace Uet_Utils
{
    public enum Process_Type { Start, Stop, Stream }
    public class Audio_Recognition_Utils : Utils_Base
    {
        private readonly SemaphoreSlim _semaphore_llim = new SemaphoreSlim(1, 1);
        private string _audio_model_file_path = "";
        private int _hw_device_number = 0;
        private string _error_message = "";
        private WaveInEvent _waveSource = new WaveInEvent();
        private WhisperProcessor _processor = null;
        private List<float> _list_float = new List<float>();
        private List<byte> _list_byte = new List<byte>();
        private DateTime _last_sound_detection = DateTime.Now;
        private TimeSpan _silence_duration_threshold = TimeSpan.FromSeconds(3);
        private float _silence_level_threshold = 30;
        private float _none_silence_ratio = 0.9f;
        private bool _is_active = false;
        private bool _is_completed = false;
        private string _recognition_text = "";

        #region public properties
        /// <summary>
        /// Thông báo lỗi bất kỳ khi nào có lỗi phát sinh.
        /// </summary>
        public string Error_Message { get => _error_message; set => _error_message = value; }
        /// <summary>
        /// Đường dẫn tới file model Whisper.
        /// </summary>
        public string Audio_Model_File_Path { get => _audio_model_file_path; set => _audio_model_file_path = value; }
        /// <summary>
        /// Index của thiết bị Audio sẽ sử dụng. (Mặc định: 0)
        /// </summary>
        public int HW_Device_Number { get => _hw_device_number; set => _hw_device_number = value; }
        /// <summary>
        /// Ngưỡng thời gian dừng tối đa để phát hiện khoảng lặng. (Mặc định 3 giây)
        /// </summary>
        public TimeSpan Silence_Duration_Threshold { get => _silence_duration_threshold; set => _silence_duration_threshold = value; }
        /// <summary>
        /// Giá trị độ lớn âm lượng tối thiểu cần đạt. Nhỏ hơn mặc định là khoảng lặng. (Mặc định: 30)
        /// </summary>
        public float Silence_Level_Threshold { get => _silence_level_threshold; set => _silence_level_threshold = value; }
        /// <summary>
        /// Tỉ lệ số lượng giá trị lớn hơn ngưỡng khoảng lặng tối thiểu. Nhỏ hơn mặc định là khoảng lặng. (Mặc định: 0.9)
        /// </summary>
        public float None_Silence_Ratio { get => _none_silence_ratio; set => _none_silence_ratio = value; }
        public bool Is_Completed { get => _is_completed; set => _is_completed = value; }
        public string Recognition_Text { get => _recognition_text; }
        #endregion
        /// <summary>
        /// Khởi tạo một đối tượng nhận dạng giọng nói.
        ///
        /// </summary>
        /// <param name="audio_model_file_path">Đường dẫn tới file model Whisper.</param>
        /// <param name="audio_hw_device_number">Index của thiết bị Audio sẽ sử dụng. (Mặc định: 0)</param>
        public Audio_Recognition_Utils(string audio_model_file_path, int hw_device_number = 0, float none_silence_ratio = 0.9f, float silence_level_threshold = 30)
        {
            this._audio_model_file_path = audio_model_file_path;
            this._hw_device_number = hw_device_number;
            this._none_silence_ratio = none_silence_ratio;
            this._silence_level_threshold = silence_level_threshold;
        }

        /// <summary>
        /// Nhận dạng giọng nói. Chuyển âm thanh sang hình ảnh sử dụng Whisper (Open AI).
        /// </summary>
        /// <param name="_list_float">Buffer Audio</param>
        /// <param name="processor">Core processer</param>
        /// <returns></returns>
        private async Task<bool> Recognition(List<float> _list_float, WhisperProcessor processor)
        {
            _recognition_text = "";
            try
            {
                _error_message = "";
                await foreach (var result in processor.ProcessAsync(_list_float.ToArray()))
                {
                    //Console.WriteLine($"{result.Start}->{result.End}: {result.Text}");
                    _recognition_text += result.Text + ", ";
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message += _ex.Message;
                return false;
            }
            finally
            {
                _is_completed = true;
            }
        }
        /// <summary>
        /// Sự kiện thu nhận âm thanh.
        /// </summary>
        /// <param name="sender"></param>
        /// <param name="e"></param>
        async void waveSource_DataAvailable(object sender, WaveInEventArgs e)
        {
            await _semaphore_llim.WaitAsync();
            try
            {
                var _value = Math.Abs(BitConverter.ToInt16(e.Buffer, (e.BytesRecorded - 2)));
                int _silence_count = 0;
                for (int index = 0; index < e.BytesRecorded; index += 2)
                {
                    short sample = BitConverter.ToInt16(e.Buffer, index);
                    if (Math.Abs(sample) < this._silence_level_threshold)
                    {
                        _silence_count++;
                    }
                }

                // If more than 90% of the samples are below the threshold, consider it as silence
                if ((e.BytesRecorded / 2) * this._none_silence_ratio > _silence_count)
                {
                    _is_completed = false;
                    _last_sound_detection = DateTime.Now;

                    byte[] buffer = e.Buffer;
                    for (int index = 0; index < e.BytesRecorded; index += 2)
                    {
                        short sample = (short)((buffer[index + 1] << 8) |
                                                buffer[index]);
                        float sample32 = sample / 32768f;
                        _list_float.Add(sample32);
                    }
                    _list_byte = _list_byte.Concat(buffer.ToList()).ToList();
                }
                else
                {
                    if (DateTime.Now - _last_sound_detection > _silence_duration_threshold && _is_active && _list_byte.Count() > 0)
                    {
                        Debug.Print("Silence detected.");
                        // stuff here
                        //var _list_temp = _list_float.ConvertAll(x => x).ToList();
                        await Recognition(_list_float, _processor);
                        _list_byte.Clear();
                        _list_float.Clear();
                        // Take action on silence detection, e.g., stop recording, log, etc.
                    }
                }
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
            }
            finally
            {
                _semaphore_llim.Release();
            }
        }
        /// <summary>
        /// Khởi tạo model, kết nối Audio HW.
        /// </summary>
        /// <returns></returns>
        public bool Init()
        {
            try
            {
                if (_waveSource != null)
                {
                    _waveSource.Dispose();
                    _waveSource = null;
                }
                if (_processor != null)
                {
                    _processor.Dispose();
                    _processor = null;
                }

                _waveSource.DeviceNumber = this._hw_device_number;
                _waveSource.WaveFormat = new WaveFormat(16000, 1);
                var whisperFactory = WhisperFactory.FromPath(this._audio_model_file_path);
                _processor = whisperFactory.CreateBuilder()
                    .WithLanguage("auto")
                    .Build();
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }
        /// <summary>
        /// Khởi chạy thu âm và nhận dạng giọng nói.
        /// </summary>
        /// <param name="process_type">Phương thức vận hành</param>
        /// <returns></returns>
        public async Task<bool> Run(Process_Type process_type)
        {
            try
            {
                switch (process_type)
                {
                    case Process_Type.Start:
                        _is_active = false;
                        _list_float.Clear();
                        _list_byte.Clear();
                        _waveSource.StartRecording();
                        _waveSource.DataAvailable += waveSource_DataAvailable;
                        break;
                    case Process_Type.Stop:
                        _is_active = false;
                        _waveSource.StopRecording();
                        _waveSource.DataAvailable -= waveSource_DataAvailable;
                        await Recognition(_list_float, _processor);
                        _list_float.Clear();
                        _list_byte.Clear();
                        break;
                    default:
                        _is_active = true;
                        _list_float.Clear();
                        _list_byte.Clear();
                        _waveSource.StartRecording();
                        _waveSource.DataAvailable += waveSource_DataAvailable;
                        break;
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public bool Dispose()
        {
            try
            {
                if (_waveSource != null)
                    _waveSource.Dispose();
                if (_processor != null)
                    _processor.Dispose();
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }
    }
}


In [ ]:
#Eye_Tracking.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Runtime.CompilerServices;
using System.Text;
using System.Threading.Tasks;
using OpenCvSharp;
using System.Threading;

namespace Uet_Utils
{
    public class Eye_Tracking : Utils_Base
    {
        private CascadeClassifier _face_cls = null;
        private CascadeClassifier _eye_cls = null;
        private VideoCapture _cam = null;
        private string _feature_face_file_path = "";
        private string _feature_eye_file_path = "";
        private string _error_message = "";
        private int _hw_device_number = 0;
        private Mat _captured_image = new Mat();
        private bool _is_active = false;

        #region Public Properties
        /// <summary>
        /// Index của thiết bị Audio sẽ sử dụng. (Mặc định: 0)
        /// </summary>
        public int HW_Device_Number { get => _hw_device_number; set => _hw_device_number = value; }
        /// <summary>
        /// Đường dẫn tới file Feature Face
        /// </summary>
        public string Feature_Face_File_Path { get => _feature_face_file_path; set => _feature_face_file_path = value; }
        /// <summary>
        /// Đường dẫn tới file Feature Eye
        /// </summary>
        public string Feature_Eye_File_Path { get => _feature_eye_file_path; set => _feature_eye_file_path = value; }
        public string Error_Message { get => _error_message; set => _error_message = value; }
        #endregion

        /// <summary>
        /// Khởi tạo một đối tượng Eye Tracker.
        /// </summary>
        /// <param name="feature_face_file_path">Đường dẫn tới file Feature Face</param>
        /// <param name="feature_eye_file_path">Đường dẫn tới file Feature Eye</param>
        public Eye_Tracking(string feature_face_file_path, string feature_eye_file_path, int hw_device_number = 0) {
            this._feature_face_file_path = feature_face_file_path;
            this._feature_eye_file_path = feature_eye_file_path;
            this._hw_device_number = hw_device_number;
        }

        public bool Is_Completed { get => throw new NotImplementedException(); set => throw new NotImplementedException(); }

        public string Recognition_Text => throw new NotImplementedException();

        public bool Dispose()
        {
            try
            {
                if (_face_cls != null && !_face_cls.IsDisposed)
                    _face_cls.Dispose();
                if (_eye_cls != null && !_eye_cls.IsDisposed)
                    _eye_cls.Dispose();
                if (_cam != null && !_cam.IsDisposed) _cam.Dispose();
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public bool Init()
        {
            try
            {
                if (_face_cls != null && !_face_cls.IsDisposed)
                    _face_cls.Dispose();
                _face_cls = new CascadeClassifier(this._feature_face_file_path);

                if (_eye_cls != null && !_eye_cls.IsDisposed)
                    _eye_cls.Dispose();
                _eye_cls = new CascadeClassifier(this._feature_eye_file_path);

                if (_cam != null && !_cam.IsDisposed)
                    _cam.Dispose();
                _cam = new VideoCapture(this._hw_device_number);
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            };
        }

        public async Task<bool> Run(Process_Type process_type)
        {
            try
            {
                switch (process_type)
                {
                    case Process_Type.Start:
                        _is_active = false;
                         return _cam.Read(_captured_image);
                    case Process_Type.Stop:
                        _is_active = false;
                        _cam.Release();
                        break;
                    default:
                        _is_active = true;
                        var _out_offset = new int[2];
                        var _old_position = new Point(0, 0);
                        var _curr_position = new Point(0, 0);
                        await Task.Run(() =>
                        {
                            while (_is_active && _cam.IsOpened())
                            {
                                _cam.Read(_captured_image);
                                Detection(_captured_image, _old_position, out _out_offset, out _curr_position);
                                _old_position = _curr_position;
                                Tools.Set_Mouse_Offset_Position(_out_offset[0], _out_offset[1]);
                                Thread.Sleep(10);
                            }
                        });
                        break;
                }
                return true;
            }
            catch (Exception _ex)
            {

                return false;
            }
        }

        public bool Detection(Mat input_mat, Point old_position, out int[] out_offset, out Point curr_point)
        {
            out_offset = new int[2];
            curr_point = new Point(0, 0);
            try
            {
                var _gray_image = new Mat(input_mat.Size(), MatType.CV_8UC1);
                Cv2.CvtColor(input_mat, _gray_image, ColorConversionCodes.BGR2GRAY);
                var _face_pos = this._face_cls.DetectMultiScale(_gray_image, 1.1, 4);
                List<Rect> _rects = new List<Rect>();
                foreach (var _rect in _face_pos)
                {
                    var _region = new Mat(_gray_image, _rect);
                    var _eye_pos = this._eye_cls.DetectMultiScale(_region);
                    _rects = _rects.Concat(_eye_pos).ToList();
                }
                curr_point.X = (int)_rects.Select(x => x.X).Average();
                curr_point.Y = (int)_rects.Select(x => x.Y).Average();
                if (old_position.X == 0 && old_position.Y == 0)
                {
                    out_offset[0] = 0;
                    out_offset[1] = 0;
                }
                else
                {
                    out_offset[0] = curr_point.X - old_position.X;
                    out_offset[1] = curr_point.Y - old_position.Y;
                }

                return true;
            }
            catch (Exception)
            {
                out_offset[0] = 0;
                out_offset[1] = 0;
                return false;
            }
        }

        public bool Detection_Example(string input_file, out int[] out_positions)
        {
            out_positions = new int[1];
            try
            {
                var input_mat = Cv2.ImRead(input_file, ImreadModes.Color);
                var _gray_image = new Mat(input_mat.Size(), MatType.CV_8UC1);
                Cv2.CvtColor(input_mat, _gray_image, ColorConversionCodes.BGR2GRAY);
                var _face_pos = this._face_cls.DetectMultiScale(_gray_image, 1.1, 4);
                foreach (var _rect in _face_pos)
                {
                    Cv2.Rectangle(input_mat, _rect, new Scalar(255, 0, 0), 3);
                    var _region = new Mat(_gray_image, _rect);
                    var _eye_pos = this._eye_cls.DetectMultiScale(_region);
                    foreach (var _sub_rect in _eye_pos)
                    {
                        Cv2.Rectangle(input_mat, _sub_rect.Add(new Point(_rect.X, _rect.Y)), new Scalar(0, 255, 0), 3);
                    }
                }
                Cv2.ImShow("img", input_mat.Resize(new Size(600, 800)));
                Cv2.WaitKey(0);
                return true;
            }
            catch (Exception)
            {

                return false;
            }
        }

        //public bool Detection_Example(Mat input_mat, out int[] out_positions)
        //{
        //    out_positions = new int[1];
        //    try
        //    {
        //        var _gray_image = new Mat(input_mat.Size(), MatType.CV_8UC1);
        //        Cv2.CvtColor(input_mat, _gray_image, ColorConversionCodes.BGR2GRAY);
        //        var _face_pos = this._face_cls.DetectMultiScale(_gray_image, 1.1, 4);
        //        foreach (var _rect in _face_pos)
        //        {
        //            Cv2.Rectangle(input_mat, _rect, new Scalar(255, 0, 0), 3);
        //            var _region = new Mat(_gray_image, _rect);
        //            var _eye_pos = this._eye_cls.DetectMultiScale(_region);
        //            foreach (var _sub_rect in _eye_pos)
        //            {
        //                Cv2.Rectangle(input_mat, _rect.Add(new Point(_rect.X, _rect.Y)), new Scalar(0, 255, 0), 3);
        //            }
        //        }
        //        Cv2.ImShow("img", input_mat);
        //        Cv2.WaitKey(0);
        //        return true;
        //    }
        //    catch (Exception)
        //    {

        //        return false;
        //    }
        //}
    }
}


In [ ]:
#Tools.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Runtime.InteropServices;
using System.Threading;

namespace Uet_Utils
{
    public enum Comparing_Method { Levenshtein, }
    public enum Mouse_Function { Left, Right, Up, Down, Left_Click, Right_Click, Double_Click }

    public static class Tools
    {
        [DllImport("user32")]
        public static extern int SetCursorPos(int x, int y);
        [DllImport("user32.dll")]
        public static extern void mouse_event(uint dwFlags, uint dx, uint dy, uint dwData, int dwExtraInfo);
        // Cấu trúc POINT để lưu trữ tọa độ chuột
        public struct POINT
        {
            public int X;
            public int Y;
        }
        [DllImport("user32.dll")]
        private static extern bool GetCursorPos(out POINT lpPoint);
        /// <summary>
        /// Thay đổi tọa độ vị trí chuột (X, Y).
        /// </summary>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        public static void Set_Mouse_Position(int x, int y)
        {
            SetCursorPos(x, y);
        }
        /// <summary>
        /// Thay đổi tọa độ vị trí chuột, tới vị trí mới.
        /// </summary>
        /// <param name="x">Offset tọa độ X</param>
        /// <param name="y">Offset tọa độ Y</param>
        public static void Set_Mouse_Offset_Position(int x, int y)
        {
            try
            {
                POINT _curr_pos = new POINT();
                _curr_pos.X = 0; _curr_pos.Y = 0;
                GetCursorPos(out _curr_pos);
                SetCursorPos(_curr_pos.X + x, _curr_pos.Y + y);
            }
            catch
            {
            }
        }
        /// <summary>
        /// Click chuột trái tại tọa độ X, Y.
        /// </summary>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        public static void Left_Mouse_Click(int x, int y)
        {
            SetCursorPos(x, y);
            mouse_event(MOUSEEVENTF_LEFTDOWN, (uint)x, (uint)y, 0, 0);
            mouse_event(MOUSEEVENTF_LEFTUP, (uint)x, (uint)y, 0, 0);
        }
        /// <summary>
        /// Click double chuột phải tại tọa độ X, Y.
        /// </summary>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        public static void Left_Mouse_Double_Click(int x, int y)
        {
            SetCursorPos(x, y);
            mouse_event(MOUSEEVENTF_LEFTDOWN, (uint)x, (uint)y, 0, 0);
            mouse_event(MOUSEEVENTF_LEFTUP, (uint)x, (uint)y, 0, 0);
            Thread.Sleep(100);
            mouse_event(MOUSEEVENTF_RIGHTDOWN, (uint)x, (uint)y, 0, 0);
            mouse_event(MOUSEEVENTF_RIGHTUP, (uint)x, (uint)y, 0, 0);
        }
        private const uint MOUSEEVENTF_LEFTDOWN = 0x02;
        private const uint MOUSEEVENTF_LEFTUP = 0x04;
        private const uint MOUSEEVENTF_RIGHTDOWN = 0x08;
        private const uint MOUSEEVENTF_RIGHTUP = 0x10;
        /// <summary>
        /// So sánh sự tương đồng giữa 2 chuỗi.
        /// </summary>
        /// <param name="s_string">Chuỗi 1</param>
        /// <param name="d_string">Chuỗi 2</param>
        /// <param name="comparing_method">Phương pháp so sánh</param>
        /// <returns></returns>
        public static double Compare_2_Word(string s_string, string d_string, Comparing_Method comparing_method = Comparing_Method.Levenshtein)
        {
            switch (comparing_method)
            {
                case Comparing_Method.Levenshtein:
                    return 1 - (double)Algorithms.Calculate(s_string, d_string) / (double)Math.Max(s_string.Length, d_string.Length);
                default:
                    return 99999;
            }

        }
    }
}


In [ ]:
#Utils_Base.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace Uet_Utils
{
    internal interface Utils_Base
    {
        /// <summary>
        /// Biến kiểm tra kết quả. Hoàn thành tiến trình.
        /// </summary>
        public bool Is_Completed { get; set; }

        /// <summary>
        /// Biến thông báo lỗi.
        /// </summary>
        public string Error_Message { get; set; }

        /// <summary>
        /// Kết quả nhận dạng âm thanh.
        /// </summary>
        public string Recognition_Text { get; }
        /// <summary>
        /// Index HW sử dụng.
        /// </summary>
        public int HW_Device_Number { get; set; }

        /// <summary>
        /// Khởi tạo đối tượng lớp tính năng.
        /// </summary>
        /// <returns></returns>
        public bool Init();
        /// <summary>
        /// Khởi chạy tính năng.
        /// </summary>
        /// <param name="process_type">Kiểu tính năng (Start, Stop, Stream)</param>
        /// <returns></returns>
        public Task<bool> Run(Process_Type process_type);

        // <summary>
        /// Xóa bỏ toàn bộ các đối tượng.
        /// </summary>
        /// <returns></returns>
        public bool Dispose();

    }
}


In [ ]:
#f
using System;
using System.Collections.Generic;
using System.ComponentModel;
using System.Data;
using System.Drawing;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Windows.Forms;
using System.Runtime.InteropServices;
using System.Threading.Tasks;
using System.Diagnostics;
using System.Threading;
using Uet_Utils;
using Uet_Utils;
using System.IO;

namespace Audio_Recognition
{

    // model https://huggingface.co/ggerganov/whisper.cpp/tree/main
    public partial class Form1 : Form
    {
        Audio_Recognition_Utils _audio_recognition = null;

        public Form1()
        {
            InitializeComponent();
            _audio_recognition = new Audio_Recognition_Utils(Txt_Model_Path.Text);
        }
        private void btn_Selected_File_Click(object sender, EventArgs e)
        {
            using (OpenFileDialog ofd = new OpenFileDialog())
            {
                if (!string.IsNullOrEmpty(Txt_Model_Path.Text))
                {
                    ofd.InitialDirectory = (new FileInfo(Txt_Model_Path.Text)).DirectoryName;
                }
                if (ofd.ShowDialog() == DialogResult.OK)
                {
                    Txt_Model_Path.Text = ofd.FileName;
                }
                _audio_recognition.Audio_Model_File_Path = Txt_Model_Path.Text;
            }
        }

        private void Btn_Model_Load_Click(object sender, EventArgs e)
        {
            try
            {
                if (_audio_recognition.Init())
                    MessageBox.Show("Model is loaded", "Information", MessageBoxButtons.OK, MessageBoxIcon.Information);
                else
                {
                    MessageBox.Show(_audio_recognition.Error_Message, "Error", MessageBoxButtons.OK, MessageBoxIcon.Error);
                }
            }
            catch (Exception _ex)
            {
                MessageBox.Show(_ex.Message, "Error", MessageBoxButtons.OK, MessageBoxIcon.Error);
            }

        }

        private async void Btn_Record_Click(object sender, EventArgs e)
        {
            if (!await _audio_recognition.Run(Process_Type.Start))
            {
                MessageBox.Show(_audio_recognition.Error_Message, "Error", MessageBoxButtons.OK, MessageBoxIcon.Error);
            }
        }

        private async void Btn_Stop_Click(object sender, EventArgs e)
        {
            _is_stream = false;
            bool _run_result = await _audio_recognition.Run(Process_Type.Stop);
            BeginInvoke(new Action(() =>
            {
                lbl_Text.Text = _audio_recognition.Recognition_Text;
                lbl_Text.Refresh();
                lbl_Text.Update();
            }));
        }
        bool _is_stream = false;
        private async void Btn_Stream_Click(object sender, EventArgs e)
        {
            await _audio_recognition.Run(Process_Type.Stream);
            await Task.Run(() => {
                while (_is_stream)
                {
                    if (_audio_recognition.Is_Completed)
                    {
                        BeginInvoke(new Action(() =>
                        {
                            lbl_Text.Text = _audio_recognition.Recognition_Text;
                            lbl_Text.Refresh();
                            lbl_Text.Update();
                        }));
                    }
                    Thread.Sleep(100);
                }
            });
        }

        private void Form1_FormClosing(object sender, FormClosingEventArgs e)
        {
            _audio_recognition.Dispose();
        }

        private void Form1_Load(object sender, EventArgs e)
        {
            //double x = Tools.Compare_2_Word("hello", "hel lo");
            //x = Tools.Compare_2_Word("left", "turn left");
            //x = Tools.Compare_2_Word("left", "hel right");
            //int[] _out = new int[1];
            //Eye_Tracking _eye_tracking = new Eye_Tracking("sources/haarcascade_frontalface_default.xml", "sources/haarcascade_eye_tree_eyeglasses.xml");
            //_eye_tracking.Init();
            //_eye_tracking.Detection_Example(@"xx.jpg", out _out);
        }

        private void button1_Click(object sender, EventArgs e)
        {
            ////WaveFileWriter.CreateWaveFile("file.wav", )
            //using (WaveFileWriter writer = new WaveFileWriter("file.wav", new WaveFormat(16000, 16, 1)))
            //{
            //    writer.Write(_list_byte.ToArray(), 0, _list_byte.Count);
            //}
        }
    }
}